In [ ]:
'''
$ git clone https://github.com/BingHongLi/line_chat_bot_tutorial.git
$ docker exec -it line-chat-bot-jupyter /bin/bash
$ pip install pandas
$ pip install scikit-learn
$ pip install mysql-connector-python
$ pip install keras
$ pip install TensorFlow
$ pip install redis

跳出container(鍵盤操作): Ctrl + P + Q
'''

In [2]:
"""
啟用伺服器基本樣板
"""
# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (LineBotApi, WebhookHandler)

# 引用無效簽章錯誤
from linebot.exceptions import (InvalidSignatureError)

# 載入json處理套件
import json

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import ( MessageEvent, TextMessage, TextSendMessage,ImageMessage)

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r'))
print(secretFileContentJson.get("channel_access_token"))
print(secretFileContentJson.get("secret_key"))
print(secretFileContentJson.get("self_user_id"))
server_url = secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
        
    return 'OK'

Hlhy/OakZWX/icqhCzMSOetHgPex89AcgVKJVAprtvx3pHC0WXIMk4sZmTy0cslymmKGzwKLp5YV8rAl5Tgge87gNm9VzmoZiwkyL9rcMRgEzKo92QZpJzp3vGpr5UKjlEzUHx8XUILszM33ag0BwAdB04t89/1O/w1cDnyilFU=
37e772fa7d48063a4cfd8669c6107046
U873be787f1bc8a4322527ab2310bc0ab


In [3]:
'''
1.刪除紀錄
2.將使用者加入使回傳值
3.結束後清空使用者紀錄

#user_profile使用者代號
#prop_key使用者回傳值
#users_cache回傳值字典
'''
#經營地的tag，拿到手
users_cache = {}


#刪除紀錄 
#回傳值字典 有 使用者ID 和 使用者回傳值
#刪除 使用者ID 和 使用者回傳值
def del_user_prop(user_profile, prop_key):
    if user_profile.user_id in users_cache and prop_key in users_cache[user_profile.user_id]:
        del users_cache[user_profile.user_id][prop_key]
        
#在字典裡創建使用者
#將使用著代號 創建一個字典
#將使用者加上回傳值
def put_user_prop(user_profile, prop_key, prop_val):
    if not user_profile.user_id in users_cache:
        users_cache[user_profile.user_id] = {}
    users_cache[user_profile.user_id][prop_key] = prop_val
    
#回傳值，並結束後重新開始
#使用者代號在回傳值字典裏面 和 回傳值在使用者者代號裡面  回傳 使用者代號及回傳值
#結束後變成空值
def get_user_prop(user_profile, prop_key):
    if user_profile.user_id in users_cache and prop_key in users_cache[user_profile.user_id]:
        return users_cache[user_profile.user_id][prop_key]
    return None

#經食材的tag，拿到手
users_cache_food = {}

#刪除紀錄 
#回傳值字典 有 使用者ID 和 使用者回傳值
#刪除 使用者ID 和 使用者回傳值
def del_user_prop_food(user_profile, prop_key):
    if user_profile.user_id in users_cache_food and prop_key in users_cache_food[user_profile.user_id]:
        del users_cache_food[user_profile.user_id][prop_key]
        
#在字典裡創建使用者
#將使用著代號 創建一個字典
#將使用者加上回傳值
def put_user_prop_food(user_profile, prop_key, prop_val):
    if not user_profile.user_id in users_cache_food:
        users_cache_food[user_profile.user_id] = {}
    users_cache_food[user_profile.user_id][prop_key] = prop_val
    
#回傳值，並結束後重新開始
#使用者代號在回傳值字典裏面 和 回傳值在使用者者代號裡面  回傳 使用者代號及回傳值
#結束後變成空值
def get_user_prop_food(user_profile, prop_key):
    if user_profile.user_id in users_cache_food and prop_key in users_cache_food[user_profile.user_id]:
        return users_cache_food[user_profile.user_id][prop_key]
    return None

In [4]:
'''
連接mySQL
'''

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector
import traceback

def recommend_camp_sites(user_contents, num):
    ret = None
    conn = None
    try:
        cfg = {
            "host": "35.229.201.139",
            "port": 3306,
            "database": "camp_db",
            "user": "iii",
            "password": "Qwer_0987"
        }
        conn = mysql.connector.connect(**cfg)
        #df = pd.read_sql("SELECT DISTINCT l.camp_title,l.camp_site,l.addr,l.latlong,l.location,l.style,l.tags,l.fb_rating,w.url FROM camp_list l JOIN camp_webs w ON l.camp_title = w.camp_title", conn)
        
        #TFIDF詞向量
        
        
        df = pd.read_sql("select * from test_url", conn)
        train_series = df["tags"] # Series
        test_series = pd.Series(user_contents) # Series
#         print('train_series',train_series)
#         print('user_contents',user_contents)
#         tokenizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words="english")
        tokenizer = TfidfVectorizer()
        train_tfidf = tokenizer.fit_transform(train_series)
        test_tfidf = tokenizer.transform(test_series)
        df_cs = pd.DataFrame(cosine_similarity(test_tfidf, train_tfidf), columns=train_series.index)
#         print('cosine_similarity(test_tfidf, train_tfidf)',cosine_similarity(test_tfidf, train_tfidf))
        df_cs["ix_test"] = test_series.index
        
        
        
        # pd.melt(): http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/
        # frame: 需要處理的數據框；
        # id_vars: 保持原樣的數據列；
        # value_vars: 需要被轉換成變量值的數據列；
        # var_name: 轉換後變量的列名；
        # value_name: 數值變量的列名。
        score = pd.melt(
            df_cs,
            id_vars = "ix_test",
            var_name = "ix_train",
            value_name = "score"
        )
        df["score"] = score["score"]
        # score遞增排序並取出指定的筆數
        df = df.sort_values(by="score")[:-(num+1):-1]
        ret = df[["camp_title", "camp_site", "tags", "score","addr","url","latlong"]]
    except Exception as e:
        print("Error: {}".format(e))
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
    return ret

In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import mysql.connector
import traceback

def recommend_recipe_sites(user_contents, num):
    ret = None
    conn = None
    try:
        cfg = {
            "host": "35.229.201.139",
            "port": 3306,
            "database": "icook_db",
            "user": "iii",
            "password": "Qwer_0987"
        }
        conn = mysql.connector.connect(**cfg)
        df = pd.read_sql("select * from final_result", conn)
        train_series = df["tags"] # Series
        test_series = pd.Series(user_contents) # Series
#         tokenizer = TfidfVectorizer(analyzer="word", ngram_range=(1, 2), min_df=0, stop_words="english")
        tokenizer = TfidfVectorizer()
        train_tfidf = tokenizer.fit_transform(train_series)
        test_tfidf = tokenizer.transform(test_series)
        df_cs = pd.DataFrame(cosine_similarity(test_tfidf, train_tfidf), columns=train_series.index)
        df_cs["ix_test"] = test_series.index
        # pd.melt(): http://shzhangji.com/cnblogs/2017/09/30/pandas-and-tidy-data/
        # frame: 需要處理的數據框；
        # id_vars: 保持原樣的數據列；
        # value_vars: 需要被轉換成變量值的數據列；
        # var_name: 轉換後變量的列名；
        # value_name: 數值變量的列名。
        score = pd.melt(
            df_cs,
            id_vars = "ix_test",
            var_name = "ix_train",
            value_name = "score"
        )
        df["score"] = score["score"]
        # score遞增排序並取出指定的筆數
#         df = df.sort_values(by="score")[:-(num+1):-1]
        df = df.sort_values(by=["score" ], ascending=[False])
        ret = df[["name", "tags", "score","url"]].head(num)
    except Exception as e:
        print("Error: {}".format(e))
        traceback.print_exc()
    finally:
        if conn:
            conn.close()
    return ret
    
recommend_result = recommend_recipe_sites(user_contents="豬肉 牛奶", num=5)
print(recommend_result)

             name      tags     score                               url
285  日式漢堡排 {露營ok}  豬肉 洋蔥 牛奶  0.861789  https://icook.tw//recipes/153137
394  自製鬆餅粉 {露營ok}        牛奶  0.831482   https://icook.tw//recipes/98719
295  法式可麗餅 {露營OK}        牛奶  0.831482  https://icook.tw//recipes/148323
347          香Q蛋餅        牛奶  0.831482  https://icook.tw//recipes/172307
325  日式可麗餅 {露營OK}        牛奶  0.831482  https://icook.tw//recipes/125205


In [6]:
'''
功能:影像辨識

點選物品辨識
    進入演算法模型cnn.5
    回傳辨識成果
'''
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

def cnn_itemindntify(img_path):
    predict = None
    items = ['吊籃', '避雷帽', '豬尾巴']
    model = load_model("./cnn.h5")
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) / 255  # 圖片: 處理0~1
    predict = items[model.predict_classes(x)[0]]
    return predict

#器材辨識UI介面
from linebot.models import (
    TemplateSendMessage,CarouselTemplate, CarouselColumn,
    PostbackTemplateAction, MessageTemplateAction, URIAction,URITemplateAction
)
from linebot.models.template import(
    ButtonsTemplate
)

itemidentify_buttons_template_message = TemplateSendMessage(
    alt_text = '器材辨識',
    template = ButtonsTemplate(
        title = '器材辨識',
        text = '選擇手機照片或拍照後再上傳',
        actions = [
          {
            "type": "uri",
            "label": "拍張照片",
            "uri": "line://nv/camera/"
          }
        ]
    )
)

Using TensorFlow backend.


In [7]:
'''
功能:文字消息TextSendMessage

第一次進入，所呈現的文字消息(封鎖再次進入也會呈現)
'''
from linebot.models import (MessageEvent, TextMessage, TextSendMessage, ImageSendMessage)

# 消息清單
reply_message_list = [
TextSendMessage(text="---------------------\n使用說明:\n---------------------\n一、營地推薦\n(點選關鍵字送出，推薦您適合的營地。)\n    \n二、拍照辨識\n(拍照立即辨識物品，並搜尋在哪裡買。)\n\n三、搜尋營地\n(選擇範圍及類型，推薦您適合的營地。)\n\n四、食譜推薦\n(選擇食材，推薦您適合的料理。)\n\n五、待確定\n\n\n六、關於我們\n(由任何問題，歡迎留言或發E-mail給我們。)")
]


In [8]:
'''
1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

接收使用者加入時
    儲存使用者資訊，產生users.txt為使用者基本資訊
    將rich_mean綁定在使用者身上
    並將第一次進入的文字消息送出(action)
'''
# 載入Follow事件
from linebot.models.events import (FollowEvent)

# 載入requests套件
import requests

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
    print("user_profile:", user_profile)
        
     # 將用戶資訊存在檔案內
    with open("./users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\r\n')
        
    # 將菜單綁定在用戶身上
    linkRichMenuId = secretFileContentJson.get("rich_menu_id")
    linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
    linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
    lineLinkMenuResponse = requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)

     # 回覆文字消息與圖片消息
    line_bot_api.reply_message(event.reply_token,reply_message_list)

In [ ]:
'''
功能:QuickReply的Button
顯示不會佔太多空間，
但要在最消息能使用
但必須要要搭配文字，才可使用
'''
from linebot.models import (
    MessageAction, URIAction,PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,QuickReply, QuickReplyButton,TextSendMessage
)

#推薦GO按鈕
postbackQRB_camp = QuickReplyButton(
    action=PostbackAction( label = "****營地推薦****", data='recommend_site')
)
postbackQRB_continue = QuickReplyButton(
    action=MessageAction( label = "****繼續挑選****", text='[::text:]繼續挑選')
)
quickReplyList = QuickReply(
    items = [postbackQRB_camp,postbackQRB_continue]
)
quickReplyList1 = QuickReply(
    items = [postbackQRB_camp]
)
quickReplyTextSendMessage = TextSendMessage(text='幫您推薦適合您的營地GO',quick_reply=quickReplyList)
quickReplyTextSendMessage1 = TextSendMessage(text='幫您推薦適合您的營地GO',quick_reply=quickReplyList1)


# 點擊後，以用戶身份發送文字消息
# MessageAction
#from selenium.webdriver import Chrome
def quick_reply_button_google_map (result_title,result_loca_url):
    postbackQRB = QuickReplyButton(
        action=MessageAction(label = "【區域地圖】",  text= "line://app/1566487232-m5e68p3P" )
    )    
    postbackQRB1 = QuickReplyButton(
        action=MessageAction(label = result_title[0], text=result_loca_url[0])
    )
    postbackQRB2 = QuickReplyButton(
        action=MessageAction( label = result_title[1], text=result_loca_url[1])
    )
    postbackQRB3 = QuickReplyButton(
        action=MessageAction( label = result_title[2], text=result_loca_url[2])
    )
    quickReplyList = QuickReply(
        items = [postbackQRB,postbackQRB1,postbackQRB2,postbackQRB3]
    )
    quickReplyTextSendMessage = TextSendMessage(text='下列按鈕可連結地圖、導航',quick_reply=quickReplyList)
    return quickReplyTextSendMessage

In [ ]:
from linebot.models.template import(
    ButtonsTemplate
)
from linebot.models import (
    TemplateSendMessage,CarouselTemplate, CarouselColumn,
    PostbackTemplateAction, MessageTemplateAction, URIAction,URITemplateAction
)
camp_search_buttons_template_message = TemplateSendMessage(
    alt_text = '營地搜尋',
    template = ButtonsTemplate(
        title = '營地搜尋',
        text = '請選擇地圖搜尋or條件搜尋。',
        actions = [
          {
            "type": "uri",
            "label": "地圖搜尋",
            "uri": "https://is.gd/9yCgXX"
          },{
            "type": "postback",
            "label": "條件搜尋",
            "text": "[::text:]條件搜尋",
            "data": "資料 2"
          }
        ]
    )
)

from linebot.models import (
    ImagemapArea, BaseSize, URIImagemapAction, MessageImagemapAction,ImagemapSendMessage
)

camp_search_imagemap_message = ImagemapSendMessage(
    base_url='https://is.gd/VKXD9X',
    alt_text='選擇類型',
    base_size=BaseSize(height=1473,width=1040)
     ,actions=[
        MessageImagemapAction(
            text="[::text:]親子同樂",
            area=ImagemapArea(x=7,y=271,width=513,height=128)
        ),
         MessageImagemapAction(
            text="[::text:]吃喝玩樂",
            area=ImagemapArea(x=5,y=404,width=513,height=132)
        ),
         MessageImagemapAction(
            text="[::text:]觀賞美景",
            area=ImagemapArea(x=4,y=540,width=513,height=132)
        )
        
    ]
)


from linebot.models import (
    ImagemapArea, BaseSize, URIImagemapAction, MessageImagemapAction,ImagemapSendMessage
)

imagemap_message = ImagemapSendMessage(
    base_url='https://is.gd/ZzkWiv',
    alt_text='選擇區域',
    base_size=BaseSize(height=1364,width=1040)
     ,actions=[
        MessageImagemapAction(
            text="[::text:]北部地區",
            area=ImagemapArea(
                x=271,y=152,width=653,height=262
            )
        ),
         MessageImagemapAction(
            text="[::text:]中部地區",
            area=ImagemapArea(
                x=45, y=426,width=598,height=366
            )
        ),
         MessageImagemapAction(
            text="[::text:]南部地區",
            area=ImagemapArea(
               x=16, y=812,width=504,height=511
            )
        ),
         MessageImagemapAction(
            text="[::text:]東部地區",
            area=ImagemapArea(
                x=648, y=562,width=390,height=733
            )
        )
        
    ]
) 




In [ ]:
'''
推薦輪播選單

參考文件
Reference: https://github.com/line/line-bot-sdk-python#templatesendmessage---buttonstemplate
'''
from linebot.models import (
    TemplateSendMessage,CarouselTemplate, CarouselColumn,
    PostbackTemplateAction, MessageTemplateAction, URIAction,URITemplateAction
)

#將推薦營地資訊，等待備取用
liff_camp_recommend_TemplateSendMessage=[]
def liff_camp_recommend_TemplateSendMessage_button():
    liff_text_message=liff_camp_recommend_TemplateSendMessage
    return liff_text_message

#將推薦營地資訊，儲存在輪播選單
def camp_recommend_TemplateSendMessage(result_title,result_loca_url,result_fb_url_list):
    camp_carousel_template_message_1 = TemplateSendMessage(
        alt_text='Carousel template',
        template=CarouselTemplate(
            columns=[
                CarouselColumn(
                    title='******【推薦排行:第一名】******',
                    text="【營地名稱】:"+result_title[0],
                    actions=[
                        URITemplateAction(
                            label='GO網站',
                            uri=result_fb_url_list[0]
                        ),
                        URITemplateAction(
                            label='GO地圖',
                            uri=result_loca_url[0]
                        )
                    ]
                ),
                 CarouselColumn(
                    title='******【推薦排行:第二名】******',
                    text="【營地名稱】:"+result_title[1],
                    actions=[
                        URITemplateAction(
                            label='GO網站',
                            uri=result_fb_url_list[1]
                        ),
                        URITemplateAction(
                            label='GO地圖',
                            uri=result_loca_url[1]
                        )
                    ]
                ),
                 CarouselColumn(
                    title='******【推薦排行:第三名】******',
                    text="【營地名稱】:"+result_title[2],
                    actions=[
                        URITemplateAction(
                            label='GO網站',
                            uri=result_fb_url_list[2]
                        ),
                        URITemplateAction(
                            label='GO地圖',
                            uri=result_loca_url[2]
                        )
                    ]
                )
            ]
        )
    )
    liff_camp_recommend_TemplateSendMessage.append(camp_carousel_template_message_1)


In [ ]:
#接推薦系統回傳值post回傳值儲存
liff_text_message_button_1=[]
liff_text_message_button_2=[]
liff_text_message_button_3=[]
def liff_camp_button_1():
    liff_text_message=liff_text_message_button_1
    return liff_text_message
def liff_camp_button_2():
    liff_text_message=liff_text_message_button_2
    return liff_text_message
def liff_camp_button_3():
    liff_text_message=liff_text_message_button_3
    return liff_text_message
def itemidentify_buttons_template_message_camp(result_ranking_list,result_title,result_fb_url_list,result_addr_list,result_loca_url):
    itemidentify_buttons_template_message_camp = TemplateSendMessage(   
            alt_text = result_ranking_list[0],
            template = ButtonsTemplate(
                title = result_ranking_list[0],
                text ="【名稱及地址】"+"\n"+result_title[0]+"\n"+result_addr_list[0] ,
                actions = [
                  {
                    "type": "uri",
                    "label": "官方網頁站",
                    "uri": result_fb_url_list[0]
                  },
                    {
                    "type": "uri",
                    "label": "前往google地圖",
                    "uri": result_loca_url[0]
                  }
                ]
            )
        )
    liff_text_message_button_1.append(itemidentify_buttons_template_message_camp)
    itemidentify_buttons_template_message_camp = TemplateSendMessage(   
        alt_text = result_ranking_list[1],
        template = ButtonsTemplate(
            title = result_ranking_list[1],
            text ="【名稱及地址】"+"\n"+result_title[1]+"\n"+result_addr_list[1] ,
            actions = [
              {
                "type": "uri",
                "label": "官方網頁站",
                "uri": result_fb_url_list[1]
              },
                {
                "type": "uri",
                "label": "前往google地圖",
                "uri": result_loca_url[1]
              }
            ]
        )
    )
    liff_text_message_button_2.append(itemidentify_buttons_template_message_camp)
    itemidentify_buttons_template_message_camp = TemplateSendMessage(   
        alt_text = result_ranking_list[2],
        template = ButtonsTemplate(
            title = result_ranking_list[2],
            text ="【名稱及地址】"+"\n"+result_title[2]+"\n"+result_addr_list[2] ,
            actions = [
              {
                "type": "uri",
                "label": "官方網頁站",
                "uri": result_fb_url_list[2]
              },
                {
                "type": "uri",
                "label": "前往google地圖",
                "uri": result_loca_url[2]
              }
            ]
        )
    )
    liff_text_message_button_3.append(itemidentify_buttons_template_message_camp)
    

In [ ]:
'''
將素材消息做成一本字典，當用戶發出相應消息時，可從此進行查找動作。
'''
import camp_ui
import food_ui
import camp_ui_2
import game


template_message_dict = {
#     "[::text:]營地推薦": survey_carousel_template_message
     "[::text:]營地推薦":[camp_ui.survey_flexCarouselSendMeesage,quickReplyTextSendMessage]
#      "[::text:]營地推薦":[camp_ui_2.survey_carousel_template_message1,quickReplyTextSendMessage]
#     ,"[::text:]繼續挑選":[camp_ui_2.survey_carousel_template_message2,quickReplyTextSendMessage1]
 
    ,"[::text:]器材辨識":itemidentify_buttons_template_message
    ,"[::text:]食譜推薦":food_ui.survey_flexCarouselSendMeesage1
    ,"[::text:]小遊戲":game.game_imagemap_message
    
    ,"[::text:]營地搜尋":camp_search_buttons_template_message
    ,"[::text:]條件搜尋":camp_search_imagemap_message
    ,"[::text:]親子同樂":imagemap_message
    ,"[::text:]吃喝玩樂":imagemap_message
    ,"[::text:]觀賞美景":imagemap_message

    ,"[::text:]第一名":liff_camp_button_1()
    ,"[::text:]第二名":liff_camp_button_2()
    ,"[::text:]第三名":liff_camp_button_3()
    ,"[::text:]輪播顯示":liff_camp_recommend_TemplateSendMessage_button()
    
#    ,"[::text:]北部地區":camp_carousel_template_message
#    ,"[::text:]中部地區":
#    ,"[::text:]南部地區":
#    ,"[::text:]東部地區":
    
#    , "[::text:]聯絡我們":camp_carousel_template_message_test

}


In [ ]:
'''
當用戶發出文字消息時，判斷文字內容是否包含[::text:]，
    若有，則從template_message_dict 內找出相關訊息
    若無，則回傳預設訊息。
'''

from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
)


# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    #顯示發送的文字
    print("event.message.text:", event.message.text)
    user_profile = line_bot_api.get_profile(event.source.user_id)
    user_profile_food = line_bot_api.get_profile(event.source.user_id)
    if (event.message.text.find('::text:') != -1):
        if "[::text:]營地推薦" == event.message.text:
            del_user_prop(user_profile, "survey_like")
        elif "[::text:]食譜推薦" == event.message.text:
            del_user_prop_food(user_profile_food, "food_like")
        elif "[::text:]器材辨識" == event.message.text:
            put_user_prop(user_profile, "do_itemidentify", True)
        elif "[::text:]下一頁" == event.message.text:
            linkRichMenuId = secretFileContentJson.get("rich_menu_id_2")
            linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
            linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
            lineLinkMenuResponse = requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
        elif "[::text:]返回上一頁" == event.message.text:
            linkRichMenuId = secretFileContentJson.get("rich_menu_id")
            linkMenuEndpoint = 'https://api.line.me/v2/bot/user/%s/richmenu/%s' % (event.source.user_id, linkRichMenuId)
            linkMenuRequestHeader = {'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFileContentJson["channel_access_token"]}
            lineLinkMenuResponse = requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
            
        line_bot_api.reply_message(
            event.reply_token,
            template_message_dict.get(event.message.text))
    else:
        pass
#         line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text="請點擊菜單上圖面，或輸入[::text:]more，取得更多幫助"))

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

from linebot.models import (
    PostbackEvent
)

'''
user_profile: {"displayName": "Nelson Chou"
, "pictureUrl": "https://profile.line-scdn.net/0hu_yDKA7dKnBEPQC6sJZVJ3h4JB0zEyw4PA5tHjZpIRBsBD0gLA9sEGA1cUVoWTh0fwg1RWBqIUdr"
, "userId": "U8633692756652c60054e28e65760bf8b"}
'''

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    #點選人的ID
    print("userId:", user_profile.user_id)
    #顯示回傳值
    print("event.postback.data:", event.postback.data)
        #如果回傳值是沒有按過的
    if (event.postback.data.find('survey_like') == 0):
        #第一次將回傳值帶入survey_like轉成list、並加上逗號
        survey_like = event.postback.data.replace("survey_like=", "").split(",")
        #顯示list
        print("營地偏好調查(by PostbackEvent):", survey_like)
        # get_user_prop上面有方法解釋
        #把舊的資訊拿出來
        ori_survey_like = get_user_prop(user_profile, "survey_like")
        #如果舊的有資訊
        if ori_survey_like:
            #回傳值變成一個list，去除重複值   
            #並增家後面的回傳值
            survey_like = list(set(survey_like).union(ori_survey_like))
        #創進依一個新的
        put_user_prop(user_profile, "survey_like", survey_like)
        #顯示目前字典裡的內容
        print("survey_like:", get_user_prop(user_profile, "survey_like"))
    
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('survey_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:
            survey_like = None     
    #查看清單        
    if (event.postback.data.find('like_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，顯示回復訊息:以輸入過回傳值
        else:
            print("使用者選擇查看清單...")
            survey_like = get_user_prop(user_profile, "survey_like")
            print("result:", survey_like)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(",".join(survey_like)))
            
#     #清除清單 -------------(沒有用)       
#     if (event.postback.data.find('clean_site') == 0):
#         #顯示目前字典裡的內容
#         survey_like = get_user_prop(user_profile, "survey_like")
#         print("survey_like:", survey_like)
#         #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
#         if survey_like == None:
#             line_bot_api.reply_message(
#                 event.reply_token,
#                 TextSendMessage("目前為空值"))
#         #如果有回傳值，顯示回復訊息:以輸入過回傳值
#         else:
#             print("使用者選擇清空清單...")
#             survey_like = None
#             print("result:目前為空值")
#             line_bot_api.reply_message(
#                 event.reply_token,
#                 TextSendMessage("已清空清單"))        
            
    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('recommend_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop(user_profile, "survey_like")
        print("survey_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:          
            #title,addr,url連在一起(測試)\n",
            recommend_camp_sites_info= recommend_camp_sites(" ".join(survey_like), 3)
            result_camp_title = recommend_camp_sites_info['camp_title']
            result_addr = recommend_camp_sites_info['addr']
            result_url = recommend_camp_sites_info['url']
            result_score= recommend_camp_sites_info['score']
            result_camp_latlong = recommend_camp_sites_info["latlong"]
            #print("result:",  result_camp_title)
            #print("result:", result_addr)
            #print("result:", result_url)
            #print("result:", result_score)
            #print("result_latlong:",list(result_camp_latlong.items()))
            
            #按下推薦後所要顯示的消息(Json)
            result_final=[]
            
            #前往liff網頁的消息
            web_map=["【前往推薦地圖】"+'\n'+'line://app/1566487232-m5e68p3P']
            result_web_map=TextSendMessage("\n".join(web_map))
            result_final.append(result_web_map)    
            
            #把推薦出來的營地變成list
            result_latlong=[]             
            result_title=[]
            result_fb_url_list=[]         
            result_addr_list=[]
            result_tags_list=[]
            result_ranking_list=[]
                  
            for n in range(0,3):
                result_list=[]
                nub=['一','二','三']
                result_list.append('****【推薦排行:第'+nub[n]+'名】****')
                result_ranking_list.append('****【推薦排行:第'+nub[n]+'名】****')
                result_list.append('【名稱】'+list(result_camp_title.items())[n][1])
                result_title.append(list(result_camp_title.items())[n][1])
                result_list.append('【地址】'+list(result_addr.items())[n][1])
                result_addr_list.append(list(result_addr.items())[n][1])
                result_list.append('【網址】'+list(result_url.items())[n][1])
                result_fb_url_list.append(list(result_url.items())[n][1])
                result_latlong.append(list(result_camp_latlong.items())[n][1])
                result_list=TextSendMessage("\n".join(result_list))

            result_loca_url=[]
            for n in range (0,3):
                location = result_latlong[n]
                location = location.replace("N","").replace("E","")
                location_list= location.split(",")
                loca_url = "https://www.google.com/maps/d/u/0/viewer?mid=1O7UA496kfLh3K-pZ9_P6dYIcTnfjzEio&ll=" +location_list[0]+ "%2C" +location_list[1]+ "&z=16"
                result_loca_url.append(loca_url)
#            print(result_loca_url)        
#            print(result_tags)
            camp_recommend_TemplateSendMessage(result_title,result_loca_url,result_fb_url_list)
#             result_final.append(camp_recommend)
            itemidentify_buttons_template_message_camp(result_ranking_list,result_title,result_fb_url_list,result_addr_list,result_loca_url)

#             google_map_url=quick_reply_button_google_map(result_title,result_loca_url)
#             result_final.append(google_map_url)
            
            put_user_prop(user_profile, "recommend_camp_sites", recommend_camp_sites_info)
            line_bot_api.reply_message(event.reply_token,result_final)

    
    
    if (event.postback.data.find('food_like') == 0):
        #第一次將回傳值帶入survey_like轉成list、並加上逗號
        survey_like = event.postback.data.replace("food_like=", "").split(",")
        #顯示list
        print("食物偏好調查(by PostbackEvent):", survey_like)
        # get_user_prop上面有方法解釋
        #把舊的資訊拿出來
        ori_survey_like = get_user_prop_food(user_profile, "food_like")
        #如果舊的有資訊
        if ori_survey_like:
            #回傳值變成一個list，去除重複值   
            #並增家後面的回傳值
            survey_like = list(set(survey_like).union(ori_survey_like))
        #創進依一個新的
        put_user_prop_food(user_profile, "food_like", survey_like)
        #顯示目前字典裡的內容
        print("food_like:", get_user_prop_food(user_profile, "food_like"))

    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('food_site') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop_food(user_profile, "food_like")
        print("food_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:
            survey_like = None     
    #查看清單        
    if (event.postback.data.find('like_food') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop_food(user_profile, "food_like")
        print("food_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，顯示回復訊息:以輸入過回傳值
        else:
            print("使用者選擇查看清單...")
            survey_like = get_user_prop_food(user_profile, "food_like")
            print("result:", survey_like)
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(",".join(survey_like)))


    #如果還沒有按過營地推薦，執行以下內容
    if (event.postback.data.find('recommend_food') == 0):
        #顯示目前字典裡的內容
        survey_like = get_user_prop_food(user_profile, "food_like")
        print("food_like:", survey_like)
        #如果還沒有數入回傳值，顯示回復訊息:請先讚過示意圖片!!!
        if survey_like == None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage("請先讚過示意圖片!!!"))
        #如果有回傳值，先抓到
        else:          
            #title,addr,url連在一起(測試)\n",
            recommend_recipe_sites_info= recommend_recipe_sites(" ".join(survey_like), 3)
            result_food_name = recommend_recipe_sites_info['name']
            result_tags = recommend_recipe_sites_info['tags']
            result_food_url = recommend_recipe_sites_info['url']
            result_food_score= recommend_recipe_sites_info['score']
            print('score',result_food_score)
            #按下推薦後所要顯示的消息(Json)
            result_food_final=[]

            for n in range(0,3):
                result_list=[]
                nub=['一','二','三']
                result_list.append('****【推薦排行:第'+nub[n]+'名】****')
                result_list.append('【名稱】'+list(result_food_name.items())[n][1])
                result_list.append('【tags】'+list(result_tags.items())[n][1])
                result_list.append('【網址】'+list(result_food_url.items())[n][1])
                result_list=TextSendMessage("\n".join(result_list))
                result_food_final.append(result_list)

            put_user_prop_food(user_profile, "recommend_recipe_sites", recommend_recipe_sites_info)
            line_bot_api.reply_message(event.reply_token,result_food_final)
    


In [ ]:
'''
若收到圖片消息時，
先回覆用戶文字消息，並從Line上將照片拿回，
在導向到購買網站。
'''
from linebot.models import (
    ImageMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if get_user_prop(user_profile, "do_itemidentify"):
        message_content = line_bot_api.get_message_content(event.message.id)
        img_path = './upload_images/'+event.message.id+'.jpg'
        with open(img_path, 'wb') as fd:
            for chunk in message_content.iter_content():
                fd.write(chunk)
        item_name = cnn_itemindntify(img_path)
        print("item_name =", item_name, ", type =", type(item_name))
        put_user_prop(user_profile, "do_itemidentify", False)
        
        shopee="https://shopee.tw/search?keyword=露營"+item_name
        feebee="https://feebee.com.tw/s/?q=露營"+item_name  
        line_bot_api.reply_message(
                event.reply_token,
                [TextSendMessage("這是~{}".format(item_name))
                 ,TextSendMessage("*******【蝦皮】*******\n"+shopee)
                 ,TextSendMessage("*******【飛比比價網】*******\n"+feebee)])


In [ ]:
from flask import render_template, request, jsonify
#
@app.route("/geomap")
def geomap():
    return render_template('geomap.html')

@app.route("/get_user_recommand")
def get_user_recommand():
    userId = request.args.get("userId", "", type=str)
    print("userId =", userId)
    #rec = get_user_prop({"user_id": userId}, "recommend_camp_sites")
    rec = users_cache[userId]["recommend_camp_sites"]
    rec_latlong = list(rec["latlong"])
    latlong = list()
    for data in rec_latlong:
        latlong.append([float(data.split(",")[0].replace("N", "")), float(data.split(",")[1].replace("E", ""))])
#     print({"camp_title": list(rec["camp_title"]), "latlong": latlong })
    
    result_loca_url=[]
    for n in range (0,3):
        location = latlong[n]
        loca_url = "https://www.google.com/maps/d/u/0/viewer?mid=1O7UA496kfLh3K-pZ9_P6dYIcTnfjzEio&ll=" +str(location[0])+ "%2C" +str(location[1])+ "&z=16"
        result_loca_url.append(loca_url)
#     print({"camp_title": list(rec["camp_title"]), "latlong": latlong,"google_map_url": result_loca_url})
    return jsonify({"camp_title": list(rec["camp_title"]), "latlong": latlong,"google_map_url": result_loca_url })
    #return jsonify({"camp_title": list(rec["camp_title"]), "latlong": latlong ,"google_map_url": result_loca_url})


In [ ]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
61.93.126.226 - - [24/May/2019 14:41:28] "GET /moo HTTP/1.1" 404 -
177.139.9.128 - - [24/May/2019 15:16:30] "GET / HTTP/1.1" 405 -
209.17.96.114 - - [24/May/2019 16:54:47] "GET / HTTP/1.0" 405 -
128.14.209.226 - - [24/May/2019 17:09:32] "GET / HTTP/1.1" 405 -
88.200.239.80 - - [24/May/2019 18:26:05] "GET / HTTP/1.1" 405 -
172.17.0.1 - - [24/May/2019 19:01:31] "GET / HTTP/1.1" 405 -
196.2.14.184 - - [24/May/2019 22:31:44] "GET / HTTP/1.1" 405 -
201.43.176.219 - - [25/May/2019 00:32:10] "GET / HTTP/1.1" 405 -
179.97.156.214 - - [25/May/2019 00:33:33] "GET / HTTP/1.1" 405 -
179.97.156.214 - - [25/May/2019 00:33:33] "GET / HTTP/1.1" 405 -
